# Requisições GodsUnchained

Aqui faremos uma busca e coleta de dados sobre as cartas do GodsUnchained

## 1. API 

## 1.1 Duplicate arguments
- Podemos passar varios argumentos relacionados a mesma coisa, eles serão como filtros que irão se acumular.

###### https://api.godsunchained.com/v0/card?god=nature&god=death

- god=nature&god=death

## 1.2 Paginação
- Podemos escolher a pagina que estamos (page), da mesma forma que podemos escolher a quantidade de resultados que irão aparecer por pagina (perPage).

###### https://api.godsunchained.com/v0/proto?page=3&perPage=20

## 1.3 Sorting (ordenando)
- Podemos ordenar nosso resultado por um fator (sort) e podemos decidir se é subindo ou descendo (order=[asc,desc,null]).

###### https://api.godsunchained.com/v0/card?sort=mana&order=asc&sort=health&order=desc

## 1.4 Informações das cartas


- God:			light, death, nature, war, magic, deception
- PackType:		rare, epic, legendary, shiny
- Rarity:		common, rare, epic, legendary, mythic
- Type:			creature, spell, weapon
- Tribe:		nether, aether, atlantean, viking, olympian, anubian, amazon
- Quality:		plain, shadow, gold, diamond
- Format:		full, card
- Set:			welcome, trial, ...

## 1.5 Sumario:

- Abaixo temos uma lista de metodos e o que ele faz:


- /card/{id}: pega a carta id
- /card: lista de cartas
- /proto/{id}: pega as informações basicas da carta id
- /proto: lista de informações de cartas

### 1.5.1 Get/card
- Abaixo temos a lista de parametros do "card".


- user:      	{address}      	get cards owned by a specific address
- rarity:      	{Rarity}      	get cards with a specific rarity
- quality:      {Quality}      	get cards with a specific quality
- god:      	{God}      	get cards with a specific god
- type:      	{Type}      	get cards with a specific type
- tribe:      	{Tribe}      	get cards with a specific tribe
- purity:      	{range}      	get cards with a particular purity
- mana:      	{range}      	get cards with a specific mana
- health:      	{range}      	get cards with a specific health
- attack:      	{range}      	get cards with a specific attack
- proto:      	{number}      	get cards with a specific prototype id
- set:          {set}           get cards with a specific temporate

In [1]:
import requests
import json
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('max_colwidth',None)

In [7]:
def basic_cards_information():
    
    base_url = 'https://api.godsunchained.com'
    version = 'v0'
    proto = 'proto?page=1'
    url = "/".join(i for i in [base_url, version, proto])
    
    try:
        req = requests.get(url)
    except Exception as erro:
        print('DEU ERRO:\t', erro)
    
    cards = json.loads(req.text)['records']
    informacoes = ['id','name','effect','god','rarity',
                   ('tribe','String'), 'mana',('attack','Int64'),
                   ('health','Int64'), 'type', 'set']
    
    resposta = []
    for card in cards:
        atributos = []
        for inf in informacoes:
            if not isinstance(inf,str):
                atributos.append(card[inf[0]][inf[1]])
                continue
            atributos.append(card[inf])
        resposta.append(atributos)
        
            
    df = pd.DataFrame(data=resposta, columns=['id','name','effect','god',
                                                'rarity','tribe', 'mana',
                                                'attack','health', 'type', 'set'])
    return df
    

In [17]:
df = basic_cards_information()
df['id'] = df['id'].astype(str)

df.to_csv('godsunchained.csv')
display(df)

,id,name,effect,god,rarity,tribe,mana,attack,health,type,set
0,1018,Black Jaguar,Confused. Regen 1.,nature,common,wild,2,3,3,creature,core
1,945,Blessing of War,"For each War creature in either void, you gain 3 favor and your opponent loses 3 favor.",war,common,,3,0,0,spell,trial
2,100920,First Pillar,Frontline. Can't attack.,light,common,structure,4,2,5,creature,trial
3,1477,Smith's Hammer,"After you attack with this relic, give +1 armor to each friendly creature.",war,rare,,5,1,2,weapon,order
4,87007,Panacean Messenger,Roar: Heal another friendly creature for 1.,neutral,common,aether,2,2,3,creature,welcome
5,1479,Echoing Fury,Blessed.<br>Give twin strike to one of your creatures.,war,common,,1,0,0,spell,order
6,1243,Tracking Bolt,"Pick one –<br>· Deal 1 damage to each creature, or<br>· Deal 3 damage to a creature.",magic,common,,2,0,0,spell,core
7,100056,Rune of Sight,Foresee 2.<br>Draw a card.,neutral,common,,2,0,0,spell,genesis
8,1020,Zealous March,"Summon up to six 2/2 Acolytes and give them protected.<br>For each Acolyte not summoned, heal your god for 4.",light,rare,,8,0,0,spell,core
9,1022,Orcish Elite,Blitz. Twin strike.,war,common,,5,3,6,creature,core


In [18]:
def economic_information(df):
    
    protos = [i for i in df['id']]
    quality = 'Meteorite'
    moeda = 'ETH'
    
    information = ['price', 'coin','volume'] # 
    cards = []
    for proto in protos:
        try:
            cards.append([get_card_price(proto, quality, moeda)[i] for i in information])
        except:
            cards.append([0,'ETH',0])
    
    df_economic = pd.DataFrame(cards, columns=[information])

    return pd.concat([df, df_economic], axis=1)
                

In [19]:
df = economic_information(df)
    

df.to_csv('godsunchained.csv')
display(df)

,id,name,effect,god,rarity,tribe,mana,attack,health,type,set,"(price,)","(coin,)","(volume,)"
0,1018,Black Jaguar,Confused. Regen 1.,nature,common,wild,2,3,3,creature,core,0.000195,ETH,200
1,945,Blessing of War,"For each War creature in either void, you gain 3 favor and your opponent loses 3 favor.",war,common,,3,0,0,spell,trial,0.000110,ETH,200
2,100920,First Pillar,Frontline. Can't attack.,light,common,structure,4,2,5,creature,trial,0.000000,ETH,0
3,1477,Smith's Hammer,"After you attack with this relic, give +1 armor to each friendly creature.",war,rare,,5,1,2,weapon,order,0.000023,ETH,200
4,87007,Panacean Messenger,Roar: Heal another friendly creature for 1.,neutral,common,aether,2,2,3,creature,welcome,0.000000,ETH,0
5,1479,Echoing Fury,Blessed.<br>Give twin strike to one of your creatures.,war,common,,1,0,0,spell,order,0.000009,ETH,200
6,1243,Tracking Bolt,"Pick one –<br>· Deal 1 damage to each creature, or<br>· Deal 3 damage to a creature.",magic,common,,2,0,0,spell,core,0.000092,ETH,200
7,100056,Rune of Sight,Foresee 2.<br>Draw a card.,neutral,common,,2,0,0,spell,genesis,0.000000,ETH,0
8,1020,Zealous March,"Summon up to six 2/2 Acolytes and give them protected.<br>For each Acolyte not summoned, heal your god for 4.",light,rare,,8,0,0,spell,core,0.000160,ETH,128
9,1022,Orcish Elite,Blitz. Twin strike.,war,common,,5,3,6,creature,core,0.000111,ETH,200


In [11]:
def get_coin_price(coin_name: str) -> float:
    """Get coin cotation in dolar

    Args:
      coin_name (str): gods-unchained, ethereum

    Returns:
      float: The dollar cotation
    """
    base_url = "https://api.coingecko.com/api"
    api_version = "v3"

    filters = [("ids", coin_name),("vs_currencies", "usd")]
    
    query = "simple/price?" + "&".join([f"{x[0]}={x[1]}" for x in filters])

    url = f"{base_url}/{api_version}/{query}"

    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
    return data[coin_name]["usd"]

In [12]:
def get_card_price(proto_id, quality, buy_token_type):

    gods_address= "0xacb3c6a43d15b907e8433077b6d38ae40936fe2c"
    base_url = "https://api.x.immutable.com/v1/orders?"
    metadata_str = "%7B%22proto%22%3A%5B%22{}%22%5D%2C%22quality%22%3A%5B%22{}%22%5D%7D".format(proto_id,quality)
    filters = [
        ("direction","asc"),
        ("include_fees","true"),
        ("order_by","buy_quantity"),
        ("page_size","200"),
        ("sell_metadata",metadata_str.replace('\\',"")),
        ("sell_token_type","ERC721"),
        ("status","active"),
        ("sell_token_address", gods_address),
    ]
    if buy_token_type == "GODS":
        filters.append(("buy_token_address","0xccc8cb5229b0ac8069c51fd58367fd1e622afd97"))
    else:
        filters.append(("buy_token_type", "ETH"))
    url = f"{base_url}" + "&".join([
      f"{x[0]}={x[1]}" for x in filters
    ])
    response = requests.get(url)
    try:
        data = response.json()

        volume = len(data['result'])
        token_id = data['result'][0]['sell']['data']['token_id']
        card_name = data['result'][0]['sell']['data']['properties']['name']
        int_price = int(data['result'][0]['buy']['data']['quantity'])
        decimals = int(data['result'][0]['buy']['data']['decimals'])
        timestamp = data['result'][0]['timestamp']
        site = f"https://market.x.immutable.com/assets/{gods_address}/{token_id}"
        return {
            "site": site,
            "card_name": card_name,
            "price": int_price / 10 ** decimals,
            "coin": buy_token_type,
            "volume": volume,
            "timestamp": timestamp
        }
    except:
        return {
            "error": response.status_code,
            "url": response.url
        }


In [1]:
proto_id = "1018"
quality = "Meteorite"
buy_token_type = "ETH" # or GODS
get_card_price(proto_id, quality, buy_token_type)

NameError: name 'get_card_price' is not defined

In [43]:
get_coin_price("ethereum")

3238.49

In [44]:
get_coin_price("gods-unchained")


2.18